In [18]:
import geopandas as gpd # Geospatial data operations
import rasterio as rio # Geospatial imagery manipulation
import rasterio.plot
import pandas as pd # Tabular data
import os
import re
import rapidfuzz # Fuzzy string matching
from tqdm.auto import tqdm # Progress bars
from tqdm.contrib.concurrent import thread_map, process_map # Parallel operations
import matplotlib # Plots
import matplotlib.pyplot as plt
import shapely # Polygon operations
#import solaris.tile as tile # Tile splitting
#import solaris.data.coco as coco
import contextlib
import io
import rasterio # Raster imagery operations
from rasterio.vrt import WarpedVRT
from rasterio import transform
from rasterio.merge import merge # Merging tiles into mosaics
from glob import glob # Finding files
from shapely.geometry import box # Bounding box operations
matplotlib.rcParams['figure.figsize'] = (20, 10)
tqdm.pandas()
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', 130)
import platform
if platform.system() == "Windows":
  prefix = "Z:/"
else:
  prefix = "ressci201900060-RNC2-Coastal/"

## Match shapefiles to images

In [19]:
filename = prefix + "Nick/filelist.txt"
if os.path.isfile(filename):
    filelist = pd.read_csv(filename, header=None).iloc[:,0]
else:
    def find_files(root):
        return pd.Series(glob(prefix + root + "**/**", recursive=True)).str.replace(prefix, "")
    filelist = pd.concat(thread_map(find_files, ["Gabrielle", "MaxarImagery", "Retrolens"]))
    filelist.to_csv(filename, index=False, header=False)
filelist

0                                                                            Gabrielle/
1                                                                      Gabrielle/Orders
2                                                                 Gabrielle/Orders/AOIs
3                               Gabrielle/Orders/AOIs/Pauanui_Tairua_07JAN2023WGS84.sbn
4                                    Gabrielle/Orders/AOIs/Pauanui_Tairua_07JAN2023.sbx
                                              ...                                      
188371    Retrolens/Wellington/PukeruaBay/Stack/PukeruaBay_01AUG1942_mosaic.jp2.aux.xml
188372    Retrolens/Wellington/PukeruaBay/Stack/PukeruaBay_22AUG1961_mosaic.tif.aux.xml
188373    Retrolens/Wellington/PukeruaBay/Stack/PukeruaBay_20SEP1980_mosaic.jp2.aux.xml
188374        Retrolens/Wellington/PukeruaBay/Stack/PukeruaBay_04APR1986_mosaic.jp2.ovr
188375                                   Retrolens/Wellington/MasterBlank_withProxy.shx
Name: 0, Length: 188376, dtype: 

In [20]:
def check_filename(filename):
    # This regex only matches shapefiles that contain something date-like in their names
    match = re.search(r'/Shorelines/.+\d{4}\w*.shp$', filename)
    return bool(match)

shapefiles = filelist[filelist.apply(check_filename)]
df = shapefiles.to_frame(name="filename")
df

,filename
29724,Gabrielle/Shorelines/Merged/Auckland/Whangapoua_19FEB2023.shp
29753,Gabrielle/Shorelines/Merged/Auckland/Oneroa_27DEC2022.shp
29757,Gabrielle/Shorelines/Merged/Auckland/PakiriNorth_14OCT2021.shp
29761,Gabrielle/Shorelines/Merged/Auckland/Tawharanui_23DEC2021.shp
29764,Gabrielle/Shorelines/Merged/Auckland/Tawharanui_01MAR2023.shp
...,...
187011,Retrolens/Wellington/KapitiSouth/Shorelines/KapitiSouth_02JAN1988.shp
187016,Retrolens/Wellington/KapitiSouth/Shorelines/KapitiSouth_06OCT1980.shp
188120,Retrolens/Wellington/PukeruaBay/Shorelines/PukeruaBay_22AUG1961.shp
188131,Retrolens/Wellington/PukeruaBay/Shorelines/PukeruaBay_Wellington_13FEB2021.shp


In [21]:
images = filelist[filelist.str.contains("/Stack/", case=False) & filelist.str.endswith((".jpg", ".jp2", ".tif"))]
images

33396     MaxarImagery/HighFreq/HawkesBay/Mahanga/Imagery/Stack/Mahanga_08NOV2019_2.tif
33397     MaxarImagery/HighFreq/HawkesBay/Mahanga/Imagery/Stack/Mahanga_08NOV2019_1.tif
33399       MaxarImagery/HighFreq/HawkesBay/Mahanga/Imagery/Stack/Mahanga_12MAR2018.tif
33402       MaxarImagery/HighFreq/HawkesBay/Mahanga/Imagery/Stack/Mahanga_31AUG2005.tif
33416       MaxarImagery/HighFreq/HawkesBay/Mahanga/Imagery/Stack/Mahanga_25DEC2015.tif
                                              ...                                      
188357            Retrolens/Wellington/PukeruaBay/Stack/PukeruaBay_19NOV1972_mosaic.jp2
188363            Retrolens/Wellington/PukeruaBay/Stack/PukeruaBay_04APR1986_mosaic.jp2
188366            Retrolens/Wellington/PukeruaBay/Stack/PukeruaBay_10NOV1977_mosaic.tif
188367            Retrolens/Wellington/PukeruaBay/Stack/PukeruaBay_22AUG1961_mosaic.tif
188370            Retrolens/Wellington/PukeruaBay/Stack/PukeruaBay_01AUG1942_mosaic.jp2
Name: 0, Length: 2485, dtype: ob

In [22]:
# When fuzzy matching, ignore these strings
# _0 will ignore leading zeros in dates
strings_to_delete = ["_mosaic", "_mosiac", "_mosaid", ".mosaic", "_cliff", "_beach", "_beachcliffsegment", "_MF.shp", "_MT.shp", "_0", "_1.tif", "_2.tif", "_3.tif", "_LDS", "_"]

def fuzz_preprocess(filename):
    for s in strings_to_delete:
        filename = filename.replace(s, "")
    # Case-insensitive
    filename = filename.lower()
    # Ignore extension
    filename = os.path.splitext(filename)[0]
    # Basename only
    filename = os.path.basename(filename)
    return filename

def get_matching_image(filename):
    dirname = os.path.dirname(filename)
    RL_dirname = dirname.replace("Stack/", "").replace("Shorelines", "Stack").replace("MaxarImagery/HighFreq", "Retrolens")
    Maxar_dirname = dirname.replace("Imagery/Shorelines", "Imagery/Stack").replace("Shorelines", "Imagery/Stack").replace("Retrolens", "MaxarImagery/HighFreq")
    Maxar_dirname_uppercase = Maxar_dirname.replace("Stack", "STACK")
    Maxar_dirname_outside_Imagery = Maxar_dirname.replace("Imagery/Stack", "Stack")
    all_files_in_folder = images[images.str.startswith((RL_dirname, Maxar_dirname, Maxar_dirname_uppercase, Maxar_dirname_outside_Imagery))]
    if len(all_files_in_folder) == 0:
        return "", 0
    match, score, index = rapidfuzz.process.extractOne(query=filename, choices=all_files_in_folder, processor=fuzz_preprocess)
    return match, score

df["matched_image"], df["match_score"] = zip(*df.filename.apply(get_matching_image))
print("Perfect matches:", sum(df.match_score == 100))
print("Imperfect matches:", sum(df.match_score < 100))
df[["filename", "matched_image", "match_score"]].sort_values(by="match_score").to_csv("shoreline_image_matching.csv", index=False)

Perfect matches: 1484
Imperfect matches: 900


In [23]:
df[df.filename.str.startswith("Gabrielle")]

,filename,matched_image,match_score
29724,Gabrielle/Shorelines/Merged/Auckland/Whangapoua_19FEB2023.shp,,0.0
29753,Gabrielle/Shorelines/Merged/Auckland/Oneroa_27DEC2022.shp,,0.0
29757,Gabrielle/Shorelines/Merged/Auckland/PakiriNorth_14OCT2021.shp,,0.0
29761,Gabrielle/Shorelines/Merged/Auckland/Tawharanui_23DEC2021.shp,,0.0
29764,Gabrielle/Shorelines/Merged/Auckland/Tawharanui_01MAR2023.shp,,0.0
...,...,...,...
32092,Gabrielle/Shorelines/Gisborne/Gisborne/Gisborne_16FEB2022.shp,,0.0
32095,Gabrielle/Shorelines/Gisborne/Gisborne/Gisborne_07MAR2023.shp,,0.0
32107,Gabrielle/Shorelines/Gisborne/TolagaBay/TolagaBay_20JAN2023.shp,,0.0
32114,Gabrielle/Shorelines/Gisborne/TolagaBay/TolagaBay_21FEB2023.shp,,0.0


In [24]:
Gabrielle = df[df.filename.str.startswith("Gabrielle")].copy()
Gabrielle_images = filelist[filelist.str.startswith("Gabrielle") & filelist.str.endswith((".jpg", ".jp2", ".tif"))]
len(Gabrielle), len(Gabrielle_images)

(295, 5786)

In [25]:
def get_matching_image(filename):
    match, score, index = rapidfuzz.process.extractOne(query=filename, choices=Gabrielle_images, processor=fuzz_preprocess)
    return match, score

Gabrielle["matched_image"], Gabrielle["match_score"] = zip(*Gabrielle.filename.apply(get_matching_image))
print("Perfect matches:", sum(Gabrielle.match_score == 100))
print("Imperfect matches:", sum(Gabrielle.match_score < 100))

Perfect matches: 218
Imperfect matches: 77


In [26]:
#pd.set_option("display.max_rows",None)
Gabrielle.filename = Gabrielle.filename.str.replace(prefix,"")
Gabrielle.matched_image = Gabrielle.matched_image.str.replace(prefix,"") 
Gabrielle.sort_values(by="match_score")

,filename,matched_image,match_score
30691,Gabrielle/Shorelines/BayofPlenty/Opotiki/BOPLINZ_Opotiki_05APR2023.shp,Gabrielle/Imagery/post_storm/Region/BayofPlenty/Opotiki/Opotiki_28FEB2023.tif,57.894737
31914,Gabrielle/Shorelines/Gisborne/TeAraroa/EastCape_18DEC2021.shp,Gabrielle/Imagery/pre-storm/Auckland/Waiheke/Onetangi_21DEC2022.tif,58.823529
30860,Gabrielle/Shorelines/BayofPlenty/EasternBoP/EasternBoP_20DEC2021.shp,Gabrielle/Imagery/pre-storm/Waikato/Matarangi/Matarangi_24DEC2022.tif,59.459459
31052,Gabrielle/Shorelines/Delivery/PostGabrielle_shorelines_21022023.shp,Gabrielle/Imagery/pre-storm/Bay of Plenty/Tauranga/tauranga-winter-01m-urban-aerial-photos-2022/BD37_500_022023.jpg,60.000000
30624,Gabrielle/Shorelines/BayofPlenty/Waihi/BOPLINZ_Waihi_05APR2023.shp,Gabrielle/Imagery/post_storm/Region/Auckland/Omaha/PNEO/OmahaPakiri_04APR2023.tif,61.538462
...,...,...,...
30463,Gabrielle/Shorelines/Waikato/Otama/Otama_14FEB2023.shp,Gabrielle/Imagery/post_storm/Region/Waikato/Otama/Otama_14FEB2023.tif,100.000000
30483,Gabrielle/Shorelines/Waikato/Whangamata/Whangamata_03JAN2022.shp,Gabrielle/Imagery/pre-storm/Waikato/Whangamata/LDS/Whangamata_03JAN2022_LDS.jp2,100.000000
30484,Gabrielle/Shorelines/Waikato/Whangamata/Whangamata_14FEB2023.shp,Gabrielle/Imagery/post_storm/Region/Waikato/Whangamata/Whangamata_14FEB2023.tif,100.000000
30184,Gabrielle/Shorelines/Merged/Northland/MarangaiBeach_15MAY2023.shp,Gabrielle/Imagery/post_storm/Region/Northland/MarangaiBeach/MarangaiBeach_15MAY2023.tif,100.000000


In [27]:
index_tiles = filelist[filelist.str.contains("Gabrielle/.+index-tiles.+.shp$")]
index_tiles

4468                      Gabrielle/Imagery/post_storm/LINZ/HawkesBay/hawkes-bay-010m-cyclone-gabrielle-aerial-photos-index-tiles-Copy.shp
10029                             Gabrielle/Imagery/post_storm/LINZ/BayofPlenty/bay-of-plenty-01m-urban-aerial-photos-index-tiles-2023.shp
11906                          Gabrielle/Imagery/post_storm/LINZ/Gisborne/gisborne-02m-cyclone-gabrielle-aerial-photos-index-tiles-202.shp
13877                          Gabrielle/Imagery/pre-storm/Waikato/TairuaPauanui/waikato-03m-rural-aerial-photos-index-tiles-2021-2023.shp
13996                               Gabrielle/Imagery/pre-storm/Waikato/LINZtemp/waikato-03m-rural-aerial-photos-index-tiles-2021-2023.shp
14137    Gabrielle/Imagery/pre-storm/Waikato/OpitoBay/waikato-03m-rural-aerial-photos-index-tiles-2021-2023/waikato-03m-rural-aerial-ph...
14194    Gabrielle/Imagery/pre-storm/Waikato/WaihiBeach/bay-of-plenty-02m-rural-aerial-photos-index-tiles-2021/bay-of-plenty-02m-rural-...
14439    Gabrielle/Imagery/

In [28]:
def read_index_tile(f):
    gdf = gpd.read_file(f)
    gdf["filename"] = f
    return gdf
Gabrielle_tiles = pd.concat(read_index_tile(prefix+f) for f in index_tiles)
len(Gabrielle_tiles)

16012

In [29]:
maybe_LDS = Gabrielle[(Gabrielle.match_score < 100) & ~Gabrielle.filename.str.contains("Delivery")].sort_values("match_score")
maybe_LDS

,filename,matched_image,match_score
30691,Gabrielle/Shorelines/BayofPlenty/Opotiki/BOPLINZ_Opotiki_05APR2023.shp,Gabrielle/Imagery/post_storm/Region/BayofPlenty/Opotiki/Opotiki_28FEB2023.tif,57.894737
31914,Gabrielle/Shorelines/Gisborne/TeAraroa/EastCape_18DEC2021.shp,Gabrielle/Imagery/pre-storm/Auckland/Waiheke/Onetangi_21DEC2022.tif,58.823529
30860,Gabrielle/Shorelines/BayofPlenty/EasternBoP/EasternBoP_20DEC2021.shp,Gabrielle/Imagery/pre-storm/Waikato/Matarangi/Matarangi_24DEC2022.tif,59.459459
30624,Gabrielle/Shorelines/BayofPlenty/Waihi/BOPLINZ_Waihi_05APR2023.shp,Gabrielle/Imagery/post_storm/Region/Auckland/Omaha/PNEO/OmahaPakiri_04APR2023.tif,61.538462
30864,Gabrielle/Shorelines/BayofPlenty/EasternBoP/EasternBoP_02JAN2022.shp,Gabrielle/Imagery/pre-storm/Waikato/TeKaroBay/TeKaroBay_03JAN2022.tif,62.857143
...,...,...,...
31340,Gabrielle/Shorelines/Auckland/Whangapoua/Whangapoua_09FEB2023.shp,Gabrielle/Imagery/post_storm/Region/Auckland/Whangapoua/Whangapoua_19FEB2023.tif,97.297297
31260,Gabrielle/Shorelines/Auckland/Managawhai/Mangawhai_29JUN2021.shp,Gabrielle/Imagery/pre-storm/Northland/Mangawhai/Mangawhai_29JUNE2021.tif,97.297297
29872,Gabrielle/Shorelines/Merged/Auckland/Whangapoua_09FEB2023.shp,Gabrielle/Imagery/post_storm/Region/Auckland/Whangapoua/Whangapoua_19FEB2023.tif,97.297297
30242,Gabrielle/Shorelines/Merged/Northland/LangsBeach_29JUN2021.shp,Gabrielle/Imagery/pre-storm/Northland/LangsBeach/LangsBeach_29JUNE2021.tif,97.435897


In [30]:
def get_resolution(filename):
  gdf = gpd.read_file(prefix+filename)
  if "LDS" not in gdf.Source.unique():
    return
  bounds = gdf.total_bounds
  intersecting_tiles = Gabrielle_tiles[Gabrielle_tiles.intersects(box(*bounds))]
  if len(intersecting_tiles) == 0:
    print(f"{filename} doesn't intersect any index tiles")
    return
  date = gdf.Date.unique()[0]
  DSASdate = gdf.DSASDate.unique()[0]
  date_options = intersecting_tiles.FLOWN.dropna().unique().tolist()
  if not date_options and "hawkes-bay-010m-cyclone-gabrielle-aerial-photos-index" in intersecting_tiles.filename.unique()[0]:
    return .1
  else:
    if DSASdate in date_options:
      date = DSASdate
    for option in date_options:
      if DSASdate in option:
        date = option
    match, score, index = rapidfuzz.process.extractOne(query=date, choices=date_options, processor=lambda s: s.replace("-", ""))
    if score != 100:
      print(f"{date} in {filename} is likely a typo for {match} ({score}% match)")
    tiles_from_this_date = intersecting_tiles[intersecting_tiles.FLOWN == match]
    assert len(tiles_from_this_date) > 0
    resolutions = tiles_from_this_date.GSDM.astype(str).str.strip("m").astype(float).unique()
    return resolutions[0]

maybe_LDS["Pixel_ER"] = maybe_LDS.filename.apply(get_resolution)
maybe_LDS[["filename", "Pixel_ER"]].dropna().to_csv("Gabrielle_LDS.csv", index=False)

2023-04-25 in Gabrielle/Shorelines/BayofPlenty/Opotiki/BOPLINZ_Opotiki_05APR2023.shp is likely a typo for 2023-04-05 (87.5% match)


In [31]:
df = pd.concat((df, Gabrielle))
df = df[df.match_score >= 100].sort_values(by="match_score")
df

,filename,matched_image,match_score
33485,MaxarImagery/HighFreq/HawkesBay/Mahanga/Shorelines/Mahanga_31AUG2005.shp,MaxarImagery/HighFreq/HawkesBay/Mahanga/Imagery/Stack/Mahanga_31AUG2005.tif,100.0
160811,Retrolens/Otago/ChrystallsBeach/Shorelines/ChrystallsBeach_28FEB1962.shp,Retrolens/Otago/ChrystallsBeach/Stack/ChrystallsBeach_28FEB1962_mosaic.jp2,100.0
160802,Retrolens/Otago/ChrystallsBeach/Shorelines/ChrystallsBeach_26FEB1946.shp,Retrolens/Otago/ChrystallsBeach/Stack/ChrystallsBeach_26FEB1946_mosaic.jp2,100.0
160801,Retrolens/Otago/ChrystallsBeach/Shorelines/ChrystallsBeach_26FEB1975.shp,Retrolens/Otago/ChrystallsBeach/Stack/ChrystallsBeach_26FEB1975_mosaic.jp2,100.0
160800,Retrolens/Otago/ChrystallsBeach/Shorelines/ChrystallsBeach_28DEC2020.shp,Retrolens/Otago/ChrystallsBeach/Stack/ChrystallsBeach_28DEC2020_LDS.tif,100.0
...,...,...,...
83662,Retrolens/HawkesBay/Waimarama/Shorelines/Waimarama_18SEP1951.shp,Retrolens/HawkesBay/Waimarama/Stack/Waimarama_18SEP1951_mosaic.jp2,100.0
83467,Retrolens/HawkesBay/OpoutamaBeach/Shorelines/OpoutamaBeach_13SEP1962.shp,Retrolens/HawkesBay/OpoutamaBeach/Stack/OpoutamaBeach_13SEP1962_mosaic.jp2,100.0
83453,Retrolens/HawkesBay/OpoutamaBeach/Shorelines/OpoutamaBeach_16JAN2003.shp,Retrolens/HawkesBay/OpoutamaBeach/Stack/OpoutamaBeach_16JAN2003_mosaic.jp2,100.0
83439,Retrolens/HawkesBay/OpoutamaBeach/Shorelines/OpoutamaBeach_10OCT1973.shp,Retrolens/HawkesBay/OpoutamaBeach/Stack/OpoutamaBeach_10OCT1973_mosaic.jp2,100.0


## Investigate metadata about the matched images

In [32]:
def get_meta(tup):
    i, row = tup
    image = rio.open(prefix + row.matched_image)
    try:
        gdf = gpd.read_file(prefix + row.filename)
        row = row.to_dict()
        row["n_lines"] = len(gdf.explode(index_parts=False))
    except: 
        print(f"Can't read{row['filename']}")
    
    row.update(image.profile)
    row["GCPs"] = len(image.gcps[0])
    row["res"] = image.res
    row["CPS"] = "CPS" in gdf.columns
    return row

metafile = "meta.csv"
if os.path.isfile(metafile):
    meta = pd.read_csv(metafile)
else:
    meta = pd.DataFrame(process_map(get_meta, df.iterrows(), total=len(df)))
    meta.to_csv(metafile, index=False)
meta

  0%|          | 0/1702 [00:00<?, ?it/s]

,filename,matched_image,match_score,n_lines,driver,dtype,nodata,width,height,count,crs,transform,blockxsize,blockysize,tiled,compress,interleave,GCPs,res,CPS,photometric
0,MaxarImagery/HighFreq/HawkesBay/Mahanga/Shorelines/Mahanga_31AUG2005.shp,MaxarImagery/HighFreq/HawkesBay/Mahanga/Imagery/Stack/Mahanga_31AUG2005.tif,100.0,2,GTiff,uint8,NaN,3975,12039,3,None,"(0.6, 0.0, 2022707.1257145917, 0.0, -0.5999999999999536, 5670278.448431859, 0.0, 0.0, 1.0)",128.0,128,True,lzw,pixel,26,"(0.6, 0.5999999999999536)",True,NaN
1,Retrolens/Otago/ChrystallsBeach/Shorelines/ChrystallsBeach_28FEB1962.shp,Retrolens/Otago/ChrystallsBeach/Stack/ChrystallsBeach_28FEB1962_mosaic.jp2,100.0,4,JP2OpenJPEG,uint16,256.0,13318,9888,3,"(proj, lat_0, lon_0, k, x_0, y_0, ellps, towgs84, units, no_defs)","(0.4214729445497887, 0.0, 1371788.78291343, 0.0, -0.4214729445498184, 4881064.116651183, 0.0, 0.0, 1.0)",1024.0,1024,True,NaN,pixel,0,"(0.4214729445497887, 0.4214729445498184)",True,NaN
2,Retrolens/Otago/ChrystallsBeach/Shorelines/ChrystallsBeach_26FEB1946.shp,Retrolens/Otago/ChrystallsBeach/Stack/ChrystallsBeach_26FEB1946_mosaic.jp2,100.0,2,JP2OpenJPEG,uint16,256.0,11398,8480,3,"(proj, lat_0, lon_0, k, x_0, y_0, ellps, towgs84, units, no_defs)","(0.5067093632678147, 0.0, 1371660.407411207, 0.0, -0.5067093632678185, 4881189.196909072, 0.0, 0.0, 1.0)",1024.0,1024,True,NaN,pixel,0,"(0.5067093632678147, 0.5067093632678185)",True,NaN
3,Retrolens/Otago/ChrystallsBeach/Shorelines/ChrystallsBeach_26FEB1975.shp,Retrolens/Otago/ChrystallsBeach/Stack/ChrystallsBeach_26FEB1975_mosaic.jp2,100.0,3,JP2OpenJPEG,uint16,256.0,3073,2249,3,"(proj, lat_0, lon_0, k, x_0, y_0, ellps, towgs84, units, no_defs)","(1.821597377107222, 0.0, 1371835.5917355102, 0.0, -1.8215973771070808, 4881106.308963457, 0.0, 0.0, 1.0)",1024.0,1024,True,NaN,pixel,0,"(1.821597377107222, 1.8215973771070808)",True,NaN
4,Retrolens/Otago/ChrystallsBeach/Shorelines/ChrystallsBeach_28DEC2020.shp,Retrolens/Otago/ChrystallsBeach/Stack/ChrystallsBeach_28DEC2020_LDS.tif,100.0,2,GTiff,uint8,NaN,18370,12177,3,None,"(0.3, 0.0, 1371939.7, 0.0, -0.2999999999999694, 4880876.100000001, 0.0, 0.0, 1.0)",128.0,128,True,lzw,pixel,4,"(0.3, 0.2999999999999694)",True,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1697,Retrolens/HawkesBay/Waimarama/Shorelines/Waimarama_18SEP1951.shp,Retrolens/HawkesBay/Waimarama/Stack/Waimarama_18SEP1951_mosaic.jp2,100.0,1,JP2OpenJPEG,uint16,256.0,3363,8576,3,"(proj, lat_0, lon_0, k, x_0, y_0, ellps, towgs84, units, no_defs)","(0.5429782858403654, 0.0, 1941506.3252280767, 0.0, -0.5429782858404023, 5589515.169659059, 0.0, 0.0, 1.0)",1024.0,1024,True,NaN,pixel,0,"(0.5429782858403654, 0.5429782858404023)",True,NaN
1698,Retrolens/HawkesBay/OpoutamaBeach/Shorelines/OpoutamaBeach_13SEP1962.shp,Retrolens/HawkesBay/OpoutamaBeach/Stack/OpoutamaBeach_13SEP1962_mosaic.jp2,100.0,3,JP2OpenJPEG,uint16,256.0,8913,7724,3,"(proj, lat_0, lon_0, k, x_0, y_0, ellps, towgs84, units, no_defs)","(0.47570421560754184, 0.0, 2017789.9928380488, 0.0, -0.4757042156075668, 5666296.99225801, 0.0, 0.0, 1.0)",1024.0,1024,True,NaN,pixel,0,"(0.47570421560754184, 0.4757042156075668)",True,NaN
1699,Retrolens/HawkesBay/OpoutamaBeach/Shorelines/OpoutamaBeach_16JAN2003.shp,Retrolens/HawkesBay/OpoutamaBeach/Stack/OpoutamaBeach_16JAN2003_mosaic.jp2,100.0,3,JP2OpenJPEG,uint16,256.0,4046,4497,3,"(proj, lat_0, lon_0, k, x_0, y_0, ellps, towgs84, units, no_defs)","(1.1211978268041494, 0.0, 2017685.0774067575, 0.0, -1.1211978268041487, 5666279.042090561, 0.0, 0.0, 1.0)",1024.0,1024,True,NaN,pixel,0,"(1.1211978268041494, 1.1211978268041487)",True,NaN
1700,Retrolens/HawkesBay/OpoutamaBeach/Shorelines/OpoutamaBeach_10OCT1973.shp,Retrolens/HawkesBay/OpoutamaBeach/Stack/OpoutamaBeach_10OCT1973_mosaic.jp2,100.0,4,JP2OpenJPEG,uint16,256.0,6469,6365,3,"(proj, lat_0, lon_0, k, x_0, y_0, ellps, towgs84, units, no_defs)","(0.7335962908908487, 0.0, 2017637.5365148883, 0.0, -0.7335962908908736, 5666213.660

In [17]:
empty = meta[meta.n_lines == 0]
empty.shape

(78, 21)

In [18]:
def get_mtime(filename):
    return pd.to_datetime(os.path.getmtime(prefix+filename), unit="s", origin="unix", utc=True).tz_convert("Pacific/Auckland")
empty["mtime"] = empty.filename.apply(get_mtime)
empty["size_bytes"] = (prefix + empty.filename).apply(os.path.getsize)
#pd.set_option("display.max_rows",None)
empty[["filename", "n_lines", "mtime", "size_bytes"]].sort_values("mtime")

/tmp/ipykernel_3458207/225968677.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  empty["mtime"] = empty.filename.apply(get_mtime)
/tmp/ipykernel_3458207/225968677.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  empty["size_bytes"] = (prefix + empty.filename).apply(os.path.getsize)


,filename,n_lines,mtime,size_bytes
1505,MaxarImagery/HighFreq/WestCoast/Ohinemaka/Shorelines/Ohinemaka_14MAR2015.shp,0,2021-06-16 12:17:29.273264896+12:00,100
1156,MaxarImagery/HighFreq/Southland/Riverton/Shorelines/Riverton_27Dec2015.shp,0,2021-06-16 12:17:29.273264896+12:00,100
1246,Retrolens/Southland/Riverton/Shorelines/Riverton_11Feb1978.shp,0,2021-06-16 12:17:29.273264896+12:00,100
1247,Retrolens/Southland/Riverton/Shorelines/Riverton_10Dec1958.shp,0,2021-06-16 12:17:29.273264896+12:00,100
1248,Retrolens/Southland/Riverton/Shorelines/Riverton_24Feb1968.shp,0,2021-06-16 12:17:29.273264896+12:00,100
...,...,...,...,...
516,Gabrielle/Shorelines/Northland/TaranuiBay/TaranuiBay_27FEB2023.shp,0,2023-09-07 12:57:53.030492672+12:00,100
966,MaxarImagery/HighFreq/Otago/TaieriBeachMouth/Shorelines/TaieriBeachMouth_25DEC2014.shp,0,2024-01-30 13:18:34.009294336+13:00,37812
967,MaxarImagery/HighFreq/Otago/TaieriBeachMouth/Shorelines/TaieriBeachMouth_01OCT2020.shp,0,2024-01-30 13:18:34.761611520+13:00,31388
964,MaxarImagery/HighFreq/Otago/TaieriBeachMouth/Shorelines/TaieriBeachMouth_11MAY2011.shp,0,2024-01-30 13:18:35.230035712+13:00,35108


In [19]:
with pd.option_context("display.max_rows", 70):
  display(empty[["filename", "n_lines", "mtime", "size_bytes"]][empty.mtime > "2021-11-23"].sort_values("mtime"))

,filename,n_lines,mtime,size_bytes
105,Retrolens/Northland/Owhata/Shorelines/Owhata_03OCT1981.shp,0,2022-01-16 15:09:57.307988992+13:00,100
975,MaxarImagery/HighFreq/Otago/Oamaru/Shorelines/Oamaru_06MAY2015.shp,0,2022-01-16 15:09:57.307988992+13:00,100
976,MaxarImagery/HighFreq/Otago/Oamaru/Shorelines/Oamaru_04APR2018.shp,0,2022-01-16 15:09:57.307988992+13:00,100
1040,MaxarImagery/HighFreq/Manawatu-Whanganui/Castlecliff/Shorelines/Castlecliff_04MAR2018.shp,0,2022-01-16 15:09:57.307988992+13:00,100
1043,MaxarImagery/HighFreq/Manawatu-Whanganui/Castlecliff/Shorelines/Castlecliff_28AUG2014.shp,0,2022-01-16 15:09:57.307988992+13:00,100
1044,MaxarImagery/HighFreq/Manawatu-Whanganui/Castlecliff/Shorelines/Castlecliff_07JUNE2016.shp,0,2022-01-16 15:09:57.307988992+13:00,100
1045,MaxarImagery/HighFreq/Manawatu-Whanganui/Castlecliff/Shorelines/Castlecliff_11APR2020.shp,0,2022-01-16 15:09:57.307988992+13:00,100
1051,MaxarImagery/HighFreq/BayOfPlenty/OhopeBeach/Shorelines/OhopeBeach_16MAY2014.shp,0,2022-01-16 15:09:57.307988992+13:00,100
1071,MaxarImagery/HighFreq/HawkesBay/TableCape/Shorelines/TableCape_04DEC2002.shp,0,2022-01-16 15:09:57.307988992+13:00,100
1072,MaxarImagery/HighFreq/HawkesBay/TableCape/Shorelines/TableCape_25DEC2015.shp,0,2022-01-16 15:09:57.307988992+13:00,100


In [20]:
meta[meta.filename.str.startswith("Gabrielle")].CPS.value_counts()

CPS
False    126
True      17
Name: count, dtype: int64

In [21]:
meta.crs.value_counts(dropna=False)

crs
PROJCS["NZGD2000 / New Zealand Transverse Mercator 2000",GEOGCS["NZGD2000",DATUM["New_Zealand_Geodetic_Datum_2000",SPHEROID["GRS 1980",6378137,298.257222101004,AUTHORITY["EPSG","7019"]],AUTHORITY["EPSG","6167"]],PRIMEM["Greenwich",0],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AUTHORITY["EPSG","4167"]],PROJECTION["Transverse_Mercator"],PARAMETER["latitude_of_origin",0],PARAMETER["central_meridian",173],PARAMETER["scale_factor",0.9996],PARAMETER["false_easting",1600000],PARAMETER["false_northing",10000000],UNIT["metre",1,AUTHORITY["EPSG","9001"]],AXIS["Easting",EAST],AXIS["Northing",NORTH],AUTHORITY["EPSG","2193"]]                          937
NaN                                                                                                                                                                                                                                                                                                                                       

In [22]:
meta.GCPs.value_counts()

GCPs
0     1082
4       96
5       91
3       77
6       66
7       62
8       31
10      20
9       15
11      11
12      10
13       8
14       7
15       4
21       4
16       4
22       3
19       2
28       2
18       2
23       1
49       1
17       1
40       1
26       1
48       1
36       1
50       1
29       1
31       1
20       1
33       1
Name: count, dtype: int64

In [23]:
meta.columns

Index(['filename', 'matched_image', 'match_score', 'n_lines', 'driver',
       'dtype', 'nodata', 'width', 'height', 'count', 'crs', 'transform',
       'blockxsize', 'blockysize', 'tiled', 'compress', 'interleave', 'GCPs',
       'res', 'CPS', 'photometric'],
      dtype='object')

In [24]:
meta.driver.value_counts()

driver
JP2OpenJPEG    902
GTiff          703
JPEG             4
Name: count, dtype: int64

In [25]:
meta["count"].value_counts()

count
3    1479
4      72
1      58
Name: count, dtype: int64

In [26]:
meta.dtype.value_counts()

dtype
uint16     937
uint8      653
int32       15
uint32       3
float32      1
Name: count, dtype: int64

In [27]:
meta.nodata.value_counts()

nodata
2.560000e+02    912
2.147484e+09     15
0.000000e+00     13
6.553500e+04      6
2.550000e+02      4
6.553600e+04      3
Name: count, dtype: int64

## Make mosaics for LINZ images

In [28]:
maybe_LDS = df[(df.match_score < 100) & df.filename.str.startswith("Retrolens")].copy()
maybe_LDS

,filename,matched_image,match_score


In [29]:
if not os.path.isfile("maybe_LDS.csv"):
    maybe_LDS.filename.to_csv("maybe_LDS.csv", index=False)

## Match shapefiles with the corresponding index tiles shapefile
- First get the bounds of every tile
- Tiles that spatially match the bounds of a drawn EOV shapefile will be used to create the corresponding mosaic

In [30]:
if os.path.isfile("tilelist.parquet"):
    tilelist = gpd.read_parquet("tilelist.parquet")
else:
    tilelist = pd.DataFrame({"filename": glob("DigitalJPGs/**/*.jpg", recursive=True)})
    tilelist["region"] = tilelist.filename.str.split("/").str[1]
    tilelist["tilename"] = tilelist.filename.str.split("/").str[-1].str.replace(".jpg", "")
    def get_bounds(f):
        return rio.open(f).bounds
    tilelist["bounds"] = thread_map(get_bounds, tilelist.filename)
    tilelist.bounds = tilelist.bounds.progress_apply(lambda b: box(*b))
    tilelist = gpd.GeoDataFrame(tilelist, geometry="bounds")
    tilelist.to_parquet("tilelist.parquet")

In [31]:
# This cell might useful for finding matches, based on geospatial correlation
for filename in tqdm(maybe_LDS.filename):
    break
    df = gpd.read_file(filename)
    if len(df) == 0:
        continue
    bounds = df.total_bounds
    intersecting_tiles = tilelist[tilelist.intersects(box(*bounds))]
    print(f"{filename} matches:\n\t{len(intersecting_tiles)} tiles from:\n\t\t{intersecting_tiles.filename}")

0it [00:00, ?it/s]

In [48]:
LDS = pd.read_csv("maybe_LDS.csv").dropna()
LDS

,filename,matched_tile_root
0,Retrolens/Nelson/BoulderBank/Shorelines/BoulderBank_16JAN2019_NEL18R.shp,DigitalJPGs/Nelson/NEL18R
18,Retrolens/Auckland/Whatipu/Shorelines/Whatipu_07APR2010.shp,DigitalJPGs/Auckland/Auckland 2010R
22,Retrolens/Auckland/Omaha/Shorelines/Omaha_04DEC2012.shp,DigitalJPGs/Auckland/RNC2 Auckland/2012
23,Retrolens/Auckland/PakiriBeach_North/Shorelines/PakiriBeach_North_06NOV2015.shp,DigitalJPGs/Northland/Northland 0.40m Rural Aerial Photos 2014-16
24,Retrolens/Auckland/TeArai/Shorelines/TeArai_06NOV2015.shp,DigitalJPGs/Northland/Northland 0.40m Rural Aerial Photos 2014-16
25,Retrolens/Auckland/Orewa/Shorelines/Orewa_08MAR2011.shp,DigitalJPGs/Auckland/RNC2 Auckland/2011
26,Retrolens/Auckland/PakiriBeach/Shorelines/PakiriBeach_04MAR2012.shp,DigitalJPGs/Auckland/RNC2 Auckland/2012
27,Retrolens/Auckland/KawakawaBay/Shorelines/KawakawaBay_03JAN2011.shp,DigitalJPGs/Auckland/RNC2 Auckland/2011
31,Retrolens/Otago/Ryans_Pipikaretu_Penguin_TeRauoneBeach/Stack/Shorelines/Ryans_Pipikaretu_Penguin_TeRauoneBeach_29JAN2019.shp,DigitalJPGs/Otago/otago-03m-rural-aerial-photos-2017-2019
36,Retrolens/Otago/Tautuku_Beach/Shorelines/Tautuku_02May2013.shp,DigitalJPGs/Southland/STH13R


For each file, create a mosaic from the corresponding tiles

In [49]:
def get_match(filename):
    match, score, index = rapidfuzz.process.extractOne(query=filename, choices=shapefiles[~shapefiles.str.contains("Old shorelines")], processor=fuzz_preprocess)
    return match, score
LDS["matched_filename"], LDS["match_score"] = zip(*LDS.filename.apply(get_match))
LDS[LDS.filename != LDS.matched_filename][["filename", "matched_filename", "match_score"]]

,filename,matched_filename,match_score
18,Retrolens/Auckland/Whatipu/Shorelines/Whatipu_07APR2010.shp,MaxarImagery/HighFreq/Auckland/Whatipu/Shorelines/Whatipu_07APR2010.shp,100.0
31,Retrolens/Otago/Ryans_Pipikaretu_Penguin_TeRauoneBeach/Stack/Shorelines/Ryans_Pipikaretu_Penguin_TeRauoneBeach_29JAN2019.shp,Retrolens/Otago/Ryans_Pipikaretu_Penguin_TeRauoneBeach/Shorelines/Ryans_Pipikaretu_Penguin_TeRauoneBeach_29JAN2019.shp,100.0
36,Retrolens/Otago/Tautuku_Beach/Shorelines/Tautuku_02May2013.shp,MaxarImagery/HighFreq/Otago/Tautuku/Shorelines/Tautuku_02May2013.shp,100.0
39,Retrolens/Otago/Aramoana/Stack/Shorelines/Aramoana_08JULY2019.shp,MaxarImagery/HighFreq/Otago/Aramoana/Shorelines/Aramoana_08JULY2019.shp,100.0
48,Retrolens/Otago/StKilda_Tomahawk_SmaillsBeach/Stack/Shorelines/StKilda_Tomahawk_SmaillsBeach_31JAN2017.shp,MaxarImagery/HighFreq/Otago/StKilda_Tomahawk_SmaillsBeach/Shorelines/StKilda_Tomahawk_SmaillsBeach_31JAN2017.shp,100.0
50,Retrolens/Otago/BoulderBeach_SandflyBay/Stack/Shorelines/BoulderBeach_SandflyBay_29JAN2019.shp,Retrolens/Otago/BoulderBeach_SandflyBay/Shorelines/BoulderBeach_SandflyBay_29JAN2019.shp,100.0
53,Retrolens/Otago/PapanuiBeach_WickliffeBay/Stack/Shorelines/PapanuiBeach_WickliffeBay_29JAN2019.shp,Retrolens/Otago/PapanuiBeach_WickliffeBay/Shorelines/PapanuiBeach_WickliffeBay_29JAN2019.shp,100.0
100,Retrolens/Bay of Plenty/OhopeBeach/Shorelines/OhopeBeach_3DEC2014.shp,Retrolens/Bay of Plenty/PortOhope/Shorelines/OhopeBeach_3DEC2014.shp,100.0
104,Retrolens/Wellington/KapitiNorth/Shorelines/NorthKapiti_15MAR2017.shp,MaxarImagery/HighFreq/Wellington/KapitiNorth/Shorelines/NorthKapiti_15MAR2017.shp,100.0


In [54]:
LDS.filename = LDS.matched_filename

In [51]:
LDS["done"] = LDS.matched_filename.apply(lambda f: os.path.isfile(prefix + f.replace(".shp", ".tif")))
LDS["done"].value_counts(dropna=False)
LDS

,filename,matched_tile_root,matched_filename,match_score,done
0,Retrolens/Nelson/BoulderBank/Shorelines/BoulderBank_16JAN2019_NEL18R.shp,DigitalJPGs/Nelson/NEL18R,Retrolens/Nelson/BoulderBank/Shorelines/BoulderBank_16JAN2019_NEL18R.shp,100.0,True
18,Retrolens/Auckland/Whatipu/Shorelines/Whatipu_07APR2010.shp,DigitalJPGs/Auckland/Auckland 2010R,MaxarImagery/HighFreq/Auckland/Whatipu/Shorelines/Whatipu_07APR2010.shp,100.0,True
22,Retrolens/Auckland/Omaha/Shorelines/Omaha_04DEC2012.shp,DigitalJPGs/Auckland/RNC2 Auckland/2012,Retrolens/Auckland/Omaha/Shorelines/Omaha_04DEC2012.shp,100.0,True
23,Retrolens/Auckland/PakiriBeach_North/Shorelines/PakiriBeach_North_06NOV2015.shp,DigitalJPGs/Northland/Northland 0.40m Rural Aerial Photos 2014-16,Retrolens/Auckland/PakiriBeach_North/Shorelines/PakiriBeach_North_06NOV2015.shp,100.0,True
24,Retrolens/Auckland/TeArai/Shorelines/TeArai_06NOV2015.shp,DigitalJPGs/Northland/Northland 0.40m Rural Aerial Photos 2014-16,Retrolens/Auckland/TeArai/Shorelines/TeArai_06NOV2015.shp,100.0,True
25,Retrolens/Auckland/Orewa/Shorelines/Orewa_08MAR2011.shp,DigitalJPGs/Auckland/RNC2 Auckland/2011,Retrolens/Auckland/Orewa/Shorelines/Orewa_08MAR2011.shp,100.0,True
26,Retrolens/Auckland/PakiriBeach/Shorelines/PakiriBeach_04MAR2012.shp,DigitalJPGs/Auckland/RNC2 Auckland/2012,Retrolens/Auckland/PakiriBeach/Shorelines/PakiriBeach_04MAR2012.shp,100.0,True
27,Retrolens/Auckland/KawakawaBay/Shorelines/KawakawaBay_03JAN2011.shp,DigitalJPGs/Auckland/RNC2 Auckland/2011,Retrolens/Auckland/KawakawaBay/Shorelines/KawakawaBay_03JAN2011.shp,100.0,True
31,Retrolens/Otago/Ryans_Pipikaretu_Penguin_TeRauoneBeach/Stack/Shorelines/Ryans_Pipikaretu_Penguin_TeRauoneBeach_29JAN2019.shp,DigitalJPGs/Otago/otago-03m-rural-aerial-photos-2017-2019,Retrolens/Otago/Ryans_Pipikaretu_Penguin_TeRauoneBeach/Shorelines/Ryans_Pipikaretu_Penguin_TeRauoneBeach_29JAN2019.shp,100.0,True
36,Retrolens/Otago/Tautuku_Beach/Shorelines/Tautuku_02May2013.shp,DigitalJPGs/Southland/STH13R,MaxarImagery/HighFreq/Otago/Tautuku/Shorelines/Tautuku_02May2013.shp,100.0,True


In [53]:
display(LDS.done.value_counts(dropna=False))

for i, row in tqdm(LDS[~LDS.done].iterrows(), total=len(LDS[~LDS.done])):
    filename = row.matched_filename
    mosaic_filename = filename.replace(".shp", ".tif")
    shapefile = gpd.read_file(filename)
    bounds = shapefile.total_bounds
    intersecting_tiles = tilelist[tilelist.intersects(box(*bounds)) & tilelist.filename.str.startswith(row.matched_tile_root)]
    tiles = list(intersecting_tiles.filename)
    print(len(tiles))
    Z, transform = merge(tiles)
    with rasterio.open(
        mosaic_filename,
        'w',
        driver='GTiff',
        height=Z.shape[1],
        width=Z.shape[2],
        count=Z.shape[0],
        dtype=Z.dtype,
        crs=shapefile.crs,
        transform=transform,
        compress='lzw',
        BIGTIFF = "IF_SAFER"
    ) as dst:
        dst.write(Z)

done
True    56
Name: count, dtype: int64

0it [00:00, ?it/s]

In [ ]:
LDS["matched_image"] = LDS.filename.str.replace(".shp", ".tif")
LDS.to_csv("LDS_matches.csv", index=False)

In [ ]:
metafile = "LDS_meta.csv"
if os.path.isfile(metafile):
    meta = pd.read_csv(metafile)
else:
    meta = pd.DataFrame(process_map(get_meta, LDS.iterrows(), total=len(LDS)))
    meta.to_csv(metafile, index=False)
meta

### Algorithm for converting polyline shapefile to polygon annotations, labelled as sea or land

In [ ]:
coastline = gpd.read_file("lds-nz-coastlines-and-islands-polygons-topo-150k-FGDB.zip!nz-coastlines-and-islands-polygons-topo-150k.gdb")

In [ ]:
# Get a random (known-good) annotation
sample = LDS.sample(1)
display(sample)
image_filename = sample.matched_image.iloc[0]
image = rio.open(image_filename)
sample_gdf = gpd.read_file(sample.filename.iloc[0])
sample_gdf

In [ ]:
def line_to_split_bbox(geo):
    bounding_box = geo.envelope
    split_bbox = shapely.ops.split(bounding_box, geo)
    return split_bbox

split_bboxes = sample_gdf.geometry.apply(line_to_split_bbox).explode(index_parts=True).reset_index()
#split_bboxes.geometry = split_bboxes.geometry.buffer(0)
split_bboxes["area"] = split_bboxes.area
split_bboxes = split_bboxes[split_bboxes.area > 1e5]
split_bboxes

In [ ]:
relevant_coastline = coastline.clip(split_bboxes.total_bounds)
split_bboxes["area_inland"] = split_bboxes.clip(relevant_coastline).area
split_bboxes["fraction_inland"] = split_bboxes.area_inland / split_bboxes.area
split_bboxes["class"] = split_bboxes.fraction_inland.apply(lambda f: "land" if f > .5 else "sea")
split_bboxes

In [ ]:
# Plot the results, and check it all looks ok
fig, ax = plt.subplots()
ax = rasterio.plot.show(image, ax=ax)

cmap = matplotlib.colors.ListedColormap(['green', 'blue'])
split_bboxes.plot(ax=ax, alpha=.5, column='class', cmap=cmap, categorical=True, legend=True, edgecolor='black')
split_bboxes.apply(lambda x: ax.annotate(text=round(x.fraction_inland, 2), xy=x.geometry.centroid.coords[0], ha='center'), axis=1)

#relevant_coastline.plot(ax=ax, alpha=.5, edgecolor="cyan")

b = split_bboxes.total_bounds
xlim = ([b[0], b[2]])
ylim = ([b[1], b[3]])
ax.set_xlim(xlim)
ax.set_ylim(ylim)